# **INTELIGENCIA DEL NEGOCIO / Business Intelligence**
## **ASSIGNMENT A4 - WORD EMBEDDING**

**INSTRUCCIONES / RECOMENDACIONES**

- Se recomienda leer con detalle la descripción de cada una de las celdas.
- Las celdas que ya tienen código, se deberán ejecutar directamente. Nota: existen alguna celdas con código que deberán completarse o parametrizarse.
- Las celdas que están vacías, se completarán con la implementación requerida en el notebook.
- No se incluirán más celdas de las establecidas en el presente notebook, por lo que la solución al mismo deberá implementarse exclusivamente en las celdas vacías.
- Scikit-Learn es un paquete muy útil para las operaciones de preprocesamiento de los datos, como estandarización, normalización, codificación, etc.
- La entrega se realizará vía Moodle. Será necesario subir la solución a este notebook con el nombre: **NOMBRE_GRUPO.ipynb**

- **Fecha de Publicación: 08/04/2024**
- **Fecha de Entrega: 14/04/2024**
- **Test: 15/04/2024**


In [ ]:
! pip install --upgrade gensim
! pip install wordcloud
! pip install tqdm

Vamos a trabajar con un dataset de titulares de noticias de la ABC en Australia durante 2020 y 2021. Vamos a realizar un análisis de text mining para tratar de saber de que temas trataron estos titulares.

In [ ]:
import pandas as pd
from wordcloud import WordCloud
from functools import reduce
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import matplotlib.pyplot as plt
from gensim.models import Word2Vec
import random
from tqdm import tqdm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
path = "abcnews_2020.csv"
pd_data = pd.read_csv(path)

In [ ]:
texts = pd_data["headline_text"].values

# Nube de palabras

Vamos a construir una nube de palabras que nos permita visualizar cuáles son las palabras más mencionadas.

Para ello, primero debemos construir un diccionario cuya clave sea la palabra y el valor sea el número de palabras que aparece.

Vamos a utilizar el <a href=https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html> CountVectorizer </a> de sklearn para ello.

In [ ]:
cv = CountVectorizer(stop_words="english")

Ajuste el cv en los textos y consiga la matriz de conteos.

Ahora, construya el diccionario. Para conseguir la palabra que representa cada columna, use el atributo vocabulary_ de count vectorizer.

In [ ]:
all_words = cv.vocabulary_
len(all_words)

Obtener el número de frecuencia de las palabras (freqs) para posteriormente utilizarlo a la hora de dibujar la nube.

Finalmente dibujamos la nube:

In [ ]:
wordcloud = WordCloud(width=800, height=500, random_state=21, max_font_size=110).generate_from_frequencies(freqs)
plt.figure(figsize=(15, 8))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis('off')
plt.title("Some frequent words used in the headlines", weight='bold', fontsize=14)
plt.show()

¿Sabrías descifrar algunos temas frecuentes en los titulares?

# Topic Modelling

Vamos a vectorizar los textos como la media de las palabras. Para ello, primero vamos a entrenar un word embedding sobre los textos.

## Word Embedding

In [ ]:
from gensim.utils import tokenize
def tokenizer(text):
  """Returns a list of tokens"""
  return tokenize(text, lowercase=True)

list(tokenizer(texts[0]))

Especificar los parámetros del modelado.

In [ ]:
SIZE_VECTORS =
WINDOW =
EPOCHS =
MIN_COUNT =

Entrenar el modelo de embeddings...

In [ ]:
we_model = Word2Vec(...)

Elija algunas palabras de la nube y muestre sus palabras más cercanas. ¿Aparecen resultados con sentido?

# Sentence Embedding

Calculamos el vector de un texto como la media de los vectores de sus palabras.

Creamos la función que genera el vector.
Hay que tener cuidado con algunos casos especiales:
- Puede que no conozcamos el vector para alguna de las palabras. En ese caso la ignoramos y calculamos la media del resto de vectores.
- En el caso de que no conozcamos el vector de ninguna de las palabras, devolveremos None.

In [ ]:
def vectorize_text(text):
  """Converts a text into a vector"""
  tokenized = tokenizer(text)

  vectors = []
  for w in tokenized:
    try:
        v = we_model.wv[w]
        vectors.append(v)
    except KeyError:
        pass
    return (text, np.mean(vectors, axis=0)) if len(vectors) else None

In [ ]:
texts_with_vectors = [vectorize_text(t) for t in tqdm(texts)]
texts_with_vectors = [x for x in texts_with_vectors if x is not None] # Removes samples on which we could not compute the vector of the sentence

In [ ]:
texts_topic_modelling, vectors = zip(*texts_with_vectors)

## Clustering

Realice clustering usando KMeans sobre los vectores generados.
Elija el número de clusters usando el método del codo.

In [ ]:
from sklearn.cluster import KMeans

## Nube de palabras para cada cluster

Para cada uno de los clusters generados:
- Tome todos los textos en ese cluster
- Dibuje la nube de palabras de dichos textos.

Debería observar que cada cluster habla de una temática distinta. ¿Puede explicar el significado de alguno de ellos?

# Reducción de vocabulario

Algunas nubes puede que hayan salido demasiado confusas porque palabras con semántica similar aparecen repetidas.

Vamos a detectar cuales tienen la misma semántica y agruparlas en una sola.

In [ ]:
from scipy.cluster.hierarchy import linkage,  dendrogram, cut_tree
from gensim.parsing.preprocessing import STOPWORDS

all_stop_words = list(STOPWORDS) + ["afl"] # appears in a lot of texts
we_model.wv.sort_by_descending_frequency() # Orders vocab in the model by descending order

all_words = np.asarray([w for w in list(we_model.wv.key_to_index.keys()) if w not in all_stop_words])


Vamos a ejecutar clustering jerárquico sobre las palabras para agrupar los sinónimos. Para elegir el threshold, tomamos unas pocas palabras y ejecutamos el algoritmo de clustering en ellas.

In [ ]:
some_words = ["covid", "coronavirus", "president", "trump", "biden", "pandemic", "market", "markets", "quarantine", "restrictions", "lockdown", "news", "updates"]
vectors_of_words = [we_model.wv[w] for w in some_words]
Z = linkage(vectors_of_words,  metric="cosine", method="complete" )

In [ ]:
plt.figure(figsize=(20, 10))
dn = dendrogram(Z, labels=some_words)
plt.grid(True)

Elija un valor de threshold adecuado que pueda agrupar palabras con la misma semántica.

In [ ]:
THRESHOLD =

In [ ]:
vectors_of_words = [we_model.wv[w] for w in all_words]
Z = linkage(vectors_of_words,  metric="cosine", method="complete")
cluster_words = cut_tree(Z, height=THRESHOLD).flatten()

In [ ]:
obtained_clusters = max(cluster_words)
obtained_clusters

Construya un diccionario que tenga por clave el número de cluster y por valor la lista de todas las palabras en dicho cluster.

Construya un segundo diccionario con las traducciones que se deben realizar.

Es decir, si la palabra "covid" debe convertirse en "coronavirus", entonces el diccionario tendra una entrada como:

{
  "covid": "coronavirus"
}

Como representante de cada cluster, tome la palabra de mayor conteo (recuerde que en all_words están ordenadas por conteo).

Compruebe algunas entradas del diccionario para comprobar que salen cosas con sentido.

In [ ]:
def replace_words_in_text(text):
  tokens = tokenizer(text)
  tokens_replacement = [dict_replacements.get(t, t) for t in tokens]
  return " ".join(tokens_replacement)

Aplique la reducción de vocabulario a cada uno de los textos. Vuelva a dibujar las nubes de cada cluster pero sobre las frases con vocabulario reducido. Comente resultados.

# Conclusiones del estudio

Detallar las principales conclusiones extraidas sobre la aplicación de técnicas de word embedding para el tratamiento de textos.

*Escribir AQUI las conclusiones*